# HOUSING KAGGLE REGRESSION PROBLEM
### Official submissions scoring - RMSLE:
First = 0.1421
### Test scoring - RMSLE:
* First = 0.1394
* Second = 0.1466





### First regression model scoring rmsle 0.1394

In [0]:
# loading dataset into the script
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_train.csv')

In [0]:
# a generic method to arrange the variables
housing = df.copy()

# starting by filling the blank spaces 
for column in housing.columns:
    if housing[column].dtype != 'object':
        housing[column].fillna(housing[column].mean(), inplace= True)
    else:
        housing[column] = housing[column].astype('str')
        housing[column].fillna(housing[column].mode(), inplace= True)

In [3]:
# checking statistics of variables after the implementation of the method above
for column in housing.columns:
    if housing[column].dtype == 'object':
        print(column, '\n', housing[column].value_counts().head(), '\n')
    else:
        print(column, '\n', housing[column].describe(), '\n')

Id 
 count    1460.000000
mean      730.500000
std       421.610009
min         1.000000
25%       365.750000
50%       730.500000
75%      1095.250000
max      1460.000000
Name: Id, dtype: float64 

MSSubClass 
 count    1460.000000
mean       56.897260
std        42.300571
min        20.000000
25%        20.000000
50%        50.000000
75%        70.000000
max       190.000000
Name: MSSubClass, dtype: float64 

MSZoning 
 RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64 

LotFrontage 
 count    1460.000000
mean       70.049958
std        22.024023
min        21.000000
25%        60.000000
50%        70.049958
75%        79.000000
max       313.000000
Name: LotFrontage, dtype: float64 

LotArea 
 count      1460.000000
mean      10516.828082
std        9981.264932
min        1300.000000
25%        7553.500000
50%        9478.500000
75%       11601.500000
max      215245.000000
Name: LotArea, dtype: float64 

Street 
 Pave    1

In [0]:
## fitting the first regression model 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score

# splitting the dependent (predictor) and dependent (target) features
x = housing.drop(['SalePrice'], axis= 1)
y = housing['SalePrice']

# applying label encoding
le = LabelEncoder()
for column in x.columns:
    if x[column].dtype == 'object':
        x[column] = le.fit_transform(x[column])

# splitting the data into train and test
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size= 0.2, random_state= 55)

In [0]:
# fitting cross validation for model evaluation
clf1_cv = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

cross_val = cross_val_score(clf1_cv, x, y, cv= 5)

In [6]:
# calling the regressor
clf1 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

# fitting the model
clf1.fit(x_treino, y_treino)

# making the prediction of target in trein set
prev = clf1.predict(x_teste)

# error evaluation
from sklearn.metrics import mean_squared_log_error
import numpy as np

erro = np.sqrt(mean_squared_log_error(y_teste, prev))
erro

0.1408533460951676

In [0]:
# making the predictions over submission set
# loading submission data
url_data = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_test.csv'
data = pd.read_csv(url_data)

In [0]:
# filling in the gaps in the same manner as used in the trein set
for column in data.columns:
    if data[column].dtype != 'object':
        data[column].fillna(data[column].mean(), inplace= True)
    else:
        data[column] = data[column].astype('str')
        data[column].fillna(data[column].mode(), inplace= True)

# using label encoder over the test set the same way as used in trein set 
le = LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le.fit_transform(data[column])

In [0]:
# df for kaggle evaluation
final = clf1.predict(data)
envio = pd.DataFrame({'Id': data['Id'], 'SalePrice': final})

In [0]:
# saving the df in a csv to be uploaded

#path = r'C:\Users\Avell\Desktop\Python\github\datasets'
#nome = '\housing_submission_1.csv'
#envio.to_csv(path+nome, index= False)

Of corse we didn't expect here anything extremely accurate for some reasons:
* The model itself was implemented just out of the box
* The approach to fill the missing values using mode to categorical features and mean in the case of numerical ones didn't take in account any exploratory analysis of the variables
* No methodology for feature engineering and in order to tune model precision by finding cerrelations or opportunities to dimensional reduction

## Model improvement attempts
### Second regression model scoring rmsle 0.1466

In [0]:
# checando com o feature importance
featimp = pd.Series(clf1.feature_importances_, index= x_teste.columns).sort_values(ascending=False)
featimp.index
feat_upper = ['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'BsmtFinSF1', 'GarageArea',
       '1stFlrSF', 'BsmtQual', '2ndFlrSF', 'YearRemodAdd', 'GarageYrBlt',
       'BsmtUnfSF', 'Neighborhood', 'LotArea', 'YearBuilt', 'TotRmsAbvGrd',
       'GarageCars', 'Id', 'CentralAir', 'OverallCond', 'MasVnrArea',
       'GarageType', 'WoodDeckSF', 'LotFrontage', 'GarageFinish',
       'BsmtExposure', 'KitchenQual', 'BsmtFinType1', 'OpenPorchSF',
       'HalfBath', 'Exterior2nd', 'YrSold', 'LotShape', 'MSZoning']

In [0]:
# novo x
x2 = x.copy()
y2 = y.copy()

# novo encoding
le2 = LabelEncoder()
for column in x2.columns:
    if x2[column].dtype == 'object':
        x2[column] = le2.fit_transform(x2[column])

In [13]:
# chamando o modelo
clf2 = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

x2_treino, x2_teste, y2_treino, y2_teste = train_test_split(x2, y2, test_size= 0.2, random_state= 11)

# ajustando um modelo
clf2.fit(x2_treino, y2_treino)

# fazendo a previsao
prev2 = clf2.predict(x2_teste)

# avaliando o erro
from sklearn.metrics import mean_squared_log_error
import numpy as np

erro2 = np.sqrt(mean_squared_log_error(y2_teste, prev2))
erro, erro2

(0.1408533460951676, 0.13921023756421952)

We can see here that simply 